In [1]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.trainers import BpeTrainer

tokenizer = Tokenizer.from_pretrained("gpt2")

c:\Github\Self-Made-Transformer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vocab = tokenizer.vocab

word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

In [3]:
print(list(word_to_idx.items())[:5])
print(len(word_to_idx))


[('shirts', 0), ('ĠFox', 1), ('Ġfiasco', 2), ('.<', 3), ('Ġcigars', 4)]
50257


In [2]:
from tokenizers import Tokenizer

# Load Huggingface GPT-2 fast BPE tokenizer
tokenizer = Tokenizer.from_pretrained("gpt2")

# Read your full text
with open('out.txt', 'r') as f:
    text = f.read()

# Define chunk size (characters)
chunk_size = 10_000_000  # About ~10MB per chunk

# Split text into smaller manageable pieces
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Tokenize each chunk
tokens = []

print(f"Tokenizing {len(chunks)} chunks...")

for idx, chunk in enumerate(chunks):
    output = tokenizer.encode(chunk)
    tokens.extend(output.tokens)

    # Progress update
    if idx % 5 == 0:
        print(f"Chunk {idx+1}/{len(chunks)} done. Total tokens so far: {len(tokens)}")

print(f"✅ Tokenization complete! Total tokens: {len(tokens)}")


Tokenizing 16 chunks...
Chunk 1/16 done. Total tokens so far: 2681563
Chunk 6/16 done. Total tokens so far: 16495997
Chunk 11/16 done. Total tokens so far: 29765540
Chunk 16/16 done. Total tokens so far: 42662204
✅ Tokenization complete! Total tokens: 42662204


In [ ]:
print(tokens[:20])